### 1. 분석환경셋팅 

In [74]:
import tensorflow as tf
from tensorflow import keras
import requests 
exec(requests.get("http://miruetoto.github.io/source/datahandling.py").text)
%load_ext rpy2.ipython

SyntaxError: invalid syntax (<string>, line 14)

In [73]:
tf.__version__
checkgpu()

GPU check 4 Keras: ['/job:localhost/replica:0/task:0/device:GPU:0']
GPU check 4 Pytorch: GeForce RTX 2080 Ti


--- 
### 2. Load data

In [4]:
mnist=keras.datasets.mnist
mnistdata=mnist.load_data()

읽힌 자료는 아래와 같은 특징이 있다. 
- mnistdata는 len=2인 튜플이다.
- mnistdata[0]에는 훈련집합이 담겨있고, mnistdata[1]에는 시험집합이 담겨있다. 
    - mnistdata[0][0]에는 train X 가 남겨있고, mnistdata[0][1]에는 train Y 가 담겨있다. 
    - mnistdata[1][0]에는 test X 가 남겨있고, mnistdata[1][1]에는 test Y 가 담겨있다. 
- mnistdata[0][0] = Xtrain = (60000,28,28) np array 
- mnistdata[0][1] = Ytrain = (60000,) np array 
- mnistdata[1][0] = Xtest = (10000,28,28) np array 
- mnistdata[1][1] = Ytest = (10000,) np array 


아래와 같은 방식으로 자료를 저장한다 

In [5]:
(Xtrain,Ytrain),(Xtest,Ytest)= mnistdata

---
### 3. 데이터가공 (reshape,astype)

In [6]:
Xtrain=Xtrain.reshape(60000,784)
Xtest=Xtest.reshape(10000,784)
Xtrain=Xtrain.astype('float32')
Xtest=Xtest.astype('float32')
Xtrain/=255
Xtest/=255
Ytrain=tf.keras.utils.to_categorical(Ytrain,10) # 10은 카테고리의 총 수, 즉 0~9까지의 숫자
Ytest=tf.keras.utils.to_categorical(Ytest,10) 

(1) reshape: MLP를 돌리기 위해서 X의 차원을 (60000,28,28) np array 에서 (60000,784) np array 로 바꾼다. 

(2) astype: X의 자료형을 'unit8'에서 'float32'로 바꿔준다. 

(3) X의 값은 0-255로 분포되어 있는데 이를 0-1사이값으로 분포하게 바꾼다.

(4) Y의 값은 원핫인코딩 

***Note: 원핫인코딩후에는 Y의 차원이 (60000,1) 에서 (60000,10)으로 바뀐다.***

***Note: 2)를 거치지 않으면 3)이 실행되지 않음. 즉 자료형이 float32일 경우에만 3)이 실행가능.***

---
### 4. 모델구축 / 아키텍쳐 설계 / 모델링

In [61]:
model1 = tf.keras.models.Sequential()
model2 = tf.keras.models.Sequential()
model1.add(keras.layers.Dense(128,input_shape=(784,),name='layer0',activation='relu'))
model1.add(keras.layers.Dense(128,name='layer1',activation='relu'))
model1.add(keras.layers.Dropout(0.3))
model1.add(keras.layers.Dense(10,name='layerL',activation='softmax'))
model1.summary()
model1.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer0 (Dense)               (None, 128)               100480    
_________________________________________________________________
layer1 (Dense)               (None, 128)               16512     
_________________________________________________________________
layerL (Dense)               (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


**몇가지노트**

(1) 파라메터 계산하는 방법: L0의 경우 입력이 (n,784)출력이 (n,128)이다. 따라서 추정할 파라메터의 수는 (784+1) * 128 = 100480 이다. 

(2) 옵티마이저 바꾸는 방법: 

model1.compile(optimizer=**'SGD'**,loss='categorical_crossentropy',metrics=['accuracy'])
model1.compile(optimizer=**'RMSprop'**,loss='categorical_crossentropy',metrics=['accuracy'])
model1.compile(optimizer=**'Adam'**,loss='categorical_crossentropy',metrics=['accuracy'])


### 5. 모델훈련 / 계수찾기 

In [62]:
model1.fit(Xtrain,Ytrain,batch_size=128,epochs=10,verbose=1, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 1s 19us/sample - loss: 0.3476 - accuracy: 0.9003 - val_loss: 0.1566 - val_accuracy: 0.9550
Epoch 2/10
48000/48000 [==============================] - 1s 15us/sample - loss: 0.1360 - accuracy: 0.9596 - val_loss: 0.1214 - val_accuracy: 0.9645
Epoch 3/10
48000/48000 [==============================] - 1s 17us/sample - loss: 0.0949 - accuracy: 0.9720 - val_loss: 0.1045 - val_accuracy: 0.9682
Epoch 4/10
48000/48000 [==============================] - 1s 16us/sample - loss: 0.0714 - accuracy: 0.9781 - val_loss: 0.1085 - val_accuracy: 0.9671
Epoch 5/10
48000/48000 [==============================] - 1s 14us/sample - loss: 0.0575 - accuracy: 0.9832 - val_loss: 0.0877 - val_accuracy: 0.9738
Epoch 6/10
48000/48000 [==============================] - 1s 18us/sample - loss: 0.0452 - accuracy: 0.9862 - val_loss: 0.1062 - val_accuracy: 0.9700
Epoch 7/10
48000/48000 [==============================] 

### 6. 모델평가 / Prediction

In [63]:
test_loss, test_acc = model1.evaluate(Xtest,Ytest,verbose=0)

In [64]:
test_loss

0.08750023055867641

In [65]:
test_acc

0.9756